In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split


df=pd.read_csv('phone_reviews.csv')

# np.percentile() is a function from the NumPy library that calculates the specified percentile of a given dataset.
percentile_25 = np.percentile(df['Ratings'], 25) 
percentile_75 = np.percentile(df['Ratings'], 75)

df['Sentiment'] = np.where(df['Ratings'] < percentile_25, 'Negative',np.where(df['Ratings'] <= percentile_75, 'Neutral', 'Positive'))


# Define a function to preprocess text data for the ML model
#  TF-IDF (Term Frequency-Inverse Document Frequency) vectorizer
def preprocess_text_ml(text_data):
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)  

    # The fit_transform method of the TF-IDF vectorizer is applied to the input text_data. This method processes the text data and converts it into a TF-IDF matrix X
    X = tfidf_vectorizer.fit_transform(text_data)
    return X

# Preprocess the text data for the ML model
X_ml = preprocess_text_ml(df['Review BODY'])

# Train an ML model (Random Forest) for sentiment analysis
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_ml, df['Sentiment'])

# Define a function to preprocess text data for the DL model
def preprocess_text_dl(text_data):
    tokenizer = Tokenizer(num_words=1000)  # You can adjust num_words as needed
    tokenizer.fit_on_texts(text_data)
    sequences = tokenizer.texts_to_sequences(text_data)
    X = pad_sequences(sequences, maxlen=100)  # You can adjust maxlen as needed
    return X, tokenizer

# Preprocess the text data for the DL model
X_dl, tokenizer = preprocess_text_dl(df['Review BODY'])

# Define and train a DL model (LSTM) for sentiment analysis
dl_model = Sequential()
dl_model.add(Embedding(input_dim=1000, output_dim=128, input_length=100))
dl_model.add(LSTM(128))
dl_model.add(Dense(3, activation='softmax'))  # 3 output classes: Negative, Neutral, Positive
dl_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert sentiment labels to one-hot encoding
sentiment_labels = pd.get_dummies(df['Sentiment']).values

# Train the DL model
X_train, X_test, y_train, y_test = train_test_split(X_dl, sentiment_labels, test_size=0.2, random_state=42)
checkpoint = ModelCheckpoint('sentiment_analysis_dl_model.h5', save_best_only=True)
dl_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpoint])

# Make predictions using the DL model
dl_predictions = dl_model.predict(X_dl)
dl_predicted_labels = np.argmax(dl_predictions, axis=1)
df['DL Sentiment Prediction'] = dl_predicted_labels

# Make predictions using the ML model
ml_predicted_labels = rf_model.predict(X_ml)
df['ML Sentiment Prediction'] = ml_predicted_labels

# Combine Ratings with other features for DL model
ratings = df['Ratings'].values
ratings = np.reshape(ratings, (-1, 1))
combined_input = np.concatenate((X_dl, ratings), axis=1)

# Define and train a DL model (LSTM) with Ratings as an additional feature
dl_model_with_ratings = Sequential()
dl_model_with_ratings.add(Input(shape=(101,)))  # 100 for text + 1 for Ratings
dl_model_with_ratings.add(Embedding(input_dim=1000, output_dim=128, input_length=101))
dl_model_with_ratings.add(LSTM(128))
dl_model_with_ratings.add(Dense(3, activation='softmax'))  # 3 output classes: Negative, Neutral, Positive
dl_model_with_ratings.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the DL model with Ratings
X_train, X_test, y_train, y_test = train_test_split(combined_input, sentiment_labels, test_size=0.2, random_state=42)
checkpoint = ModelCheckpoint('sentiment_analysis_dl_model_with_ratings.h5', save_best_only=True)
dl_model_with_ratings.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpoint])

# Load the DL model with Ratings (optional, if you want to use the saved model for predictions)
# dl_model_with_ratings = load_model('sentiment_analysis_dl_model_with_ratings.h5')

# Make predictions using the DL model with Ratings
dl_ratings_predictions = dl_model_with_ratings.predict(combined_input)
dl_ratings_predicted_labels = np.argmax(dl_ratings_predictions, axis=1)
df['DL Sentiment Prediction with Ratings'] = dl_ratings_predicted_labels

# Print the results
print(df[['Review BODY', 'Ratings', 'Sentiment', 'ML Sentiment Prediction', 'DL Sentiment Prediction', 'DL Sentiment Prediction with Ratings']])


Epoch 1/10
12/12 [==============================] - 2s 117ms/step - loss: 0.9540 - accuracy: 0.6513 - val_loss: 0.9143 - val_accuracy: 0.6211
Epoch 2/10
 2/12 [====>.........................] - ETA: 0s - loss: 0.8020 - accuracy: 0.6953

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 1s 94ms/step - loss: 0.7970 - accuracy: 0.6803 - val_loss: 0.8023 - val_accuracy: 0.6474
Epoch 3/10
12/12 [==============================] - 1s 93ms/step - loss: 0.7213 - accuracy: 0.6974 - val_loss: 0.7509 - val_accuracy: 0.6474
Epoch 4/10
12/12 [==============================] - 1s 91ms/step - loss: 0.6269 - accuracy: 0.7355 - val_loss: 0.6850 - val_accuracy: 0.7263
Epoch 5/10
12/12 [==============================] - 1s 92ms/step - loss: 0.5165 - accuracy: 0.7961 - val_loss: 0.6223 - val_accuracy: 0.7895
Epoch 6/10
12/12 [==============================] - 1s 95ms/step - loss: 0.5285 - accuracy: 0.8303 - val_loss: 0.5902 - val_accuracy: 0.7789
Epoch 7/10
12/12 [==============================] - 1s 91ms/step - loss: 0.4240 - accuracy: 0.8461 - val_loss: 0.5677 - val_accuracy: 0.8158
Epoch 8/10
12/12 [==============================] - 1s 91ms/step - loss: 0.3588 - accuracy: 0.8816 - val_loss: 0.5397 - val_accuracy: 0.8158
Epoch 9/10
12/12 [======

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


12/12 [==============================] - 1s 94ms/step - loss: 0.8144 - accuracy: 0.6724 - val_loss: 0.8108 - val_accuracy: 0.6526
Epoch 3/10
12/12 [==============================] - 1s 97ms/step - loss: 0.7316 - accuracy: 0.6987 - val_loss: 0.7715 - val_accuracy: 0.6526
Epoch 4/10
12/12 [==============================] - 1s 92ms/step - loss: 0.6424 - accuracy: 0.7145 - val_loss: 0.6632 - val_accuracy: 0.7000
Epoch 5/10
12/12 [==============================] - 1s 93ms/step - loss: 0.5316 - accuracy: 0.7803 - val_loss: 0.5653 - val_accuracy: 0.7579
Epoch 6/10
12/12 [==============================] - 1s 93ms/step - loss: 0.4336 - accuracy: 0.8461 - val_loss: 0.5124 - val_accuracy: 0.8000
Epoch 7/10
12/12 [==============================] - 1s 93ms/step - loss: 0.3487 - accuracy: 0.8539 - val_loss: 0.4532 - val_accuracy: 0.8579
Epoch 8/10
12/12 [==============================] - 1s 100ms/step - loss: 0.2757 - accuracy: 0.8868 - val_loss: 0.4074 - val_accuracy: 0.8526
Epoch 9/10
12/12 [=====

In [14]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Mobile Name,Model No,Ratings,Total Ratings,Total Reviews,Star Wise Rating,Review Title,Review Body,Review BODY,Sentiment,DL Sentiment Prediction,ML Sentiment Prediction,DL Sentiment Prediction with Ratings
0,0,0,"POCO M6 Pro 5G (Power Black, 128 GB)",MZB0EPSIN,4.3,"1,279",100,"['799', '272', '81', '29', '98']","['Good quality product', 'Really Nice', 'Value...","[""['Best phone in this price range'"", "" 'Thank...",.Best phone in this price range.Thank you Flip...,Neutral,1,Neutral,1
1,1,1,"POCO M6 Pro 5G (Forest Green, 64 GB)",MZB0EQJIN,4.4,"5,123",316,"['3,585', '896', '261', '112', '269']","['Worth the money', 'Wonderful', 'Awesome', 'M...","[""['Very nice 👍 phone 📱'"", "" '🔥 premium look ""...",".Very nice 👍 phone 📱.🔥 premium look ""."".Displa...",Positive,2,Positive,2
2,2,2,"POCO M6 Pro 5G (Power Black, 64 GB)",MZB0EPRIN,4.4,NAN,NAN,"['3,585', '896', '261', '112', '269']","['Worth the money', 'Wonderful', 'Awesome', 'M...","[""['Very nice 👍 phone 📱'"", "" '🔥 premium look ""...",".Very nice 👍 phone 📱.🔥 premium look ""."".Displa...",Positive,2,Positive,2
3,3,3,"POCO C51 (Power Black, 64 GB)",MZB0E6DIN,4.1,"58,550","3,213","['33,513', '11,048', '5,212', '2,362', '6,415']","['Best in the market!', 'Simply awesome', 'Nic...","[""['Very Good phone 📱'"", "" 'Justify with the p...",.Very Good phone 📱.Justify with the price Good...,Neutral,1,Neutral,1
4,4,4,"POCO C51 (Royal Blue, 64 GB)",MZB0DXKIN,4.1,"58,550","3,213","['33,513', '11,048', '5,212', '2,362', '6,415']","['Best in the market!', 'Simply awesome', 'Nic...","[""['Very Good phone 📱'"", "" 'Justify with the p...",.Very Good phone 📱.Justify with the price Good...,Neutral,1,Neutral,1


In [15]:
# Define a mapping from numerical values to sentiment labels
sentiment_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

# Update the columns with sentiment labels
df['DL Sentiment Prediction'] = df['DL Sentiment Prediction'].replace(sentiment_mapping)
df['ML Sentiment Prediction'] = df['ML Sentiment Prediction'].replace(sentiment_mapping)

# Print the updated DataFrame
print(df[['Review BODY', 'Ratings', 'Sentiment', 'ML Sentiment Prediction', 'DL Sentiment Prediction', 'DL Sentiment Prediction with Ratings']])


                                           Review BODY  Ratings Sentiment  \
0    .Best phone in this price range.Thank you Flip...      4.3   Neutral   
1    .Very nice 👍 phone 📱.🔥 premium look ".".Displa...      4.4  Positive   
2    .Very nice 👍 phone 📱.🔥 premium look ".".Displa...      4.4  Positive   
3    .Very Good phone 📱.Justify with the price Good...      4.1   Neutral   
4    .Very Good phone 📱.Justify with the price Good...      4.1   Neutral   
..                                                 ...      ...       ...   
945  .Super phone and am so impressed with this tha...      4.2   Neutral   
946  .m very disappointed when I played ..  apart f...      4.1   Neutral   
947                  .Good product 👍.Good.Nice camera.      4.3   Neutral   
948  .Very  Excellent  Phone👍👍👌.I like this order b...      4.2   Neutral   
949  .Goodgood.amazing phone nice camera.Good camer...      4.2   Neutral   

    ML Sentiment Prediction DL Sentiment Prediction  \
0                   

In [17]:
df.head(20)

,Unnamed: 0.1,Unnamed: 0,Mobile Name,Model No,Ratings,Total Ratings,Total Reviews,Star Wise Rating,Review Title,Review Body,Review BODY,Sentiment,DL Sentiment Prediction,ML Sentiment Prediction,DL Sentiment Prediction with Ratings
0,0,0,"POCO M6 Pro 5G (Power Black, 128 GB)",MZB0EPSIN,4.3,"1,279",100,"['799', '272', '81', '29', '98']","['Good quality product', 'Really Nice', 'Value...","[""['Best phone in this price range'"", "" 'Thank...",.Best phone in this price range.Thank you Flip...,Neutral,Neutral,Neutral,1
1,1,1,"POCO M6 Pro 5G (Forest Green, 64 GB)",MZB0EQJIN,4.4,"5,123",316,"['3,585', '896', '261', '112', '269']","['Worth the money', 'Wonderful', 'Awesome', 'M...","[""['Very nice 👍 phone 📱'"", "" '🔥 premium look ""...",".Very nice 👍 phone 📱.🔥 premium look ""."".Displa...",Positive,Positive,Positive,2
2,2,2,"POCO M6 Pro 5G (Power Black, 64 GB)",MZB0EPRIN,4.4,NAN,NAN,"['3,585', '896', '261', '112', '269']","['Worth the money', 'Wonderful', 'Awesome', 'M...","[""['Very nice 👍 phone 📱'"", "" '🔥 premium look ""...",".Very nice 👍 phone 📱.🔥 premium look ""."".Displa...",Positive,Positive,Positive,2
3,3,3,"POCO C51 (Power Black, 64 GB)",MZB0E6DIN,4.1,"58,550","3,213","['33,513', '11,048', '5,212', '2,362', '6,415']","['Best in the market!', 'Simply awesome', 'Nic...","[""['Very Good phone 📱'"", "" 'Justify with the p...",.Very Good phone 📱.Justify with the price Good...,Neutral,Neutral,Neutral,1
4,4,4,"POCO C51 (Royal Blue, 64 GB)",MZB0DXKIN,4.1,"58,550","3,213","['33,513', '11,048', '5,212', '2,362', '6,415']","['Best in the market!', 'Simply awesome', 'Nic...","[""['Very Good phone 📱'"", "" 'Justify with the p...",.Very Good phone 📱.Justify with the price Good...,Neutral,Neutral,Neutral,1
5,5,5,"REDMI 12 (Moonstone Silver, 128 GB)",MZB0EQLIN,4.1,"3,146",275,"['1,858', '621', '236', '105', '326']","['Just wow!', 'Terrific purchase', 'Really Nic...","['[""The phone is too good. I prefer to buy thi...",".[""The phone is too good. I prefer to buy this...",Neutral,Neutral,Neutral,1
6,6,6,"POCO C50 (Royal Blue, 32 GB)",MZB0D3DIN|MZB0D3CIN,4.2,"53,917","2,600","['33,560', '9,456', '4,564', '2,051', '4,286']","['Classy product', 'Pretty good', 'Pretty good...","[""['Very good product ❤️👍'"", "" 'Good product w...",.Very good product ❤️👍.Good product with good ...,Neutral,Neutral,Neutral,1
7,7,7,"POCO C55 (Power Black, 128 GB)",MZB0DDKIN,4.1,"39,709","2,331","['23,091', '8,122', '3,426', '1,466', '3,604']","['Nice product', 'Mind-blowing purchase', 'Goo...","[""['Camera is average as compared to earlier m...",.Camera is average as compared to earlier mode...,Neutral,Neutral,Neutral,1
8,8,8,"REDMI 12 (Pastel Blue, 128 GB)",MZB0EQMIN,4.1,"3,146",275,"['1,858', '621', '236', '105', '326']","['Just wow!', 'Terrific purchase', 'Really Nic...","['[""The phone is too good. I prefer to buy thi...",".[""The phone is too good. I prefer to buy this...",Neutral,Neutral,Neutral,1
9,9,9,"realme 11x 5G (Midnight Black, 128 GB)",RMX3785,0.0,NAN,NAN,"['0', '0', '0', '0', '0']",NaN,NaN,....,Negative,Negative,Negative,0


In [26]:
ls=df[df['DL Sentiment Prediction']=='Neutral'].index

In [27]:
len(ls)

663

In [28]:
ls1=df[df['DL Sentiment Prediction']=='Positive'].index

In [29]:
len(ls1)

180

In [30]:
ls2=df[df['DL Sentiment Prediction']=='Negative'].index

In [31]:
len(ls2)

107

In [36]:
df['Review BODY'][ls2[-3]]

'.Good camera.t buy this phone. If u have lots of waste money then buy this products this company is totally wrost and also flipkart is also wrost.. Because Flipkart is Delete all bad reviews".t buy it please".The phone does not have Google Playstore and does not support the required apps but this was never informed while purchasing the phone. And Flipkart does not let you return the product. Ridiculous experience. Do not buy.'

In [37]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Mobile Name,Model No,Ratings,Total Ratings,Total Reviews,Star Wise Rating,Review Title,Review Body,Review BODY,Sentiment,DL Sentiment Prediction,ML Sentiment Prediction,DL Sentiment Prediction with Ratings
0,0,0,"POCO M6 Pro 5G (Power Black, 128 GB)",MZB0EPSIN,4.3,"1,279",100,"['799', '272', '81', '29', '98']","['Good quality product', 'Really Nice', 'Value...","[""['Best phone in this price range'"", "" 'Thank...",.Best phone in this price range.Thank you Flip...,Neutral,Neutral,Neutral,1
1,1,1,"POCO M6 Pro 5G (Forest Green, 64 GB)",MZB0EQJIN,4.4,"5,123",316,"['3,585', '896', '261', '112', '269']","['Worth the money', 'Wonderful', 'Awesome', 'M...","[""['Very nice 👍 phone 📱'"", "" '🔥 premium look ""...",".Very nice 👍 phone 📱.🔥 premium look ""."".Displa...",Positive,Positive,Positive,2
2,2,2,"POCO M6 Pro 5G (Power Black, 64 GB)",MZB0EPRIN,4.4,NAN,NAN,"['3,585', '896', '261', '112', '269']","['Worth the money', 'Wonderful', 'Awesome', 'M...","[""['Very nice 👍 phone 📱'"", "" '🔥 premium look ""...",".Very nice 👍 phone 📱.🔥 premium look ""."".Displa...",Positive,Positive,Positive,2
3,3,3,"POCO C51 (Power Black, 64 GB)",MZB0E6DIN,4.1,"58,550","3,213","['33,513', '11,048', '5,212', '2,362', '6,415']","['Best in the market!', 'Simply awesome', 'Nic...","[""['Very Good phone 📱'"", "" 'Justify with the p...",.Very Good phone 📱.Justify with the price Good...,Neutral,Neutral,Neutral,1
4,4,4,"POCO C51 (Royal Blue, 64 GB)",MZB0DXKIN,4.1,"58,550","3,213","['33,513', '11,048', '5,212', '2,362', '6,415']","['Best in the market!', 'Simply awesome', 'Nic...","[""['Very Good phone 📱'"", "" 'Justify with the p...",.Very Good phone 📱.Justify with the price Good...,Neutral,Neutral,Neutral,1


In [38]:
df2=pd.read_csv('Final_Final.csv')

In [39]:
df2.head()

,Unnamed: 0,Mobile Name,Images,Price,In The Box,Model Number,Color,Display Size,Operating System,Internal Storage,...,Battery Capacity,Rating,Star Wise Rating,Total_Ratings,Total_Reviews,Review Title,REVIEW BODY,DL Sentiment Pred,ML Sentiment Pred,Company Name
0,0,"POCO M6 Pro 5G (Power Black, 128 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹12,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPSIN,Power Black,17.25 cm (6.79 inch),Android 13,128 GB,...,5000 mAh,4.3,"['799', '272', '81', '29', '98']","1,279",100,"['Good quality product', 'Really Nice', 'Value...",best phone in this price range thank you flipk...,Positive,Positive,POCO
1,1,"POCO M6 Pro 5G (Forest Green, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹10,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EQJIN,Forest Green,17.25 cm (6.79 inch),Android 13,64 GB,...,5000 mAh,4.4,"['3,585', '896', '261', '112', '269']","5,123",316,"['Worth the money', 'Wonderful', 'Awesome', 'M...",very nice phone premium look display quality,Positive,Positive,POCO
2,2,"POCO M6 Pro 5G (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹10,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPRIN,Power Black,17.25 cm (6.79 inch),Android 13,64 GB,...,5000 mAh,4.4,"['3,585', '896', '261', '112', '269']",NAN,NAN,"['Worth the money', 'Wonderful', 'Awesome', 'M...",very nice phone premium look display quality,Positive,Positive,POCO
3,3,"POCO C51 (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0E6DIN,Power Black,16.56 cm (6.52 inch),Android 13,64 GB,...,5000 mAh,4.1,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,Neutral,Neutral,POCO
4,4,"POCO C51 (Royal Blue, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0DXKIN,Royal Blue,16.56 cm (6.52 inch),Android 13,64 GB,...,5000 mAh,4.1,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,Neutral,Neutral,POCO


In [40]:
df2['DL Sentiment Prediction']=df['DL Sentiment Prediction']
df2['ML Sentiment Prediction']=df['ML Sentiment Prediction']

In [41]:
df2.head()

,Unnamed: 0,Mobile Name,Images,Price,In The Box,Model Number,Color,Display Size,Operating System,Internal Storage,...,Star Wise Rating,Total_Ratings,Total_Reviews,Review Title,REVIEW BODY,DL Sentiment Pred,ML Sentiment Pred,Company Name,DL Sentiment Prediction,ML Sentiment Prediction
0,0,"POCO M6 Pro 5G (Power Black, 128 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹12,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPSIN,Power Black,17.25 cm (6.79 inch),Android 13,128 GB,...,"['799', '272', '81', '29', '98']","1,279",100,"['Good quality product', 'Really Nice', 'Value...",best phone in this price range thank you flipk...,Positive,Positive,POCO,Neutral,Neutral
1,1,"POCO M6 Pro 5G (Forest Green, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹10,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EQJIN,Forest Green,17.25 cm (6.79 inch),Android 13,64 GB,...,"['3,585', '896', '261', '112', '269']","5,123",316,"['Worth the money', 'Wonderful', 'Awesome', 'M...",very nice phone premium look display quality,Positive,Positive,POCO,Positive,Positive
2,2,"POCO M6 Pro 5G (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹10,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPRIN,Power Black,17.25 cm (6.79 inch),Android 13,64 GB,...,"['3,585', '896', '261', '112', '269']",NAN,NAN,"['Worth the money', 'Wonderful', 'Awesome', 'M...",very nice phone premium look display quality,Positive,Positive,POCO,Positive,Positive
3,3,"POCO C51 (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0E6DIN,Power Black,16.56 cm (6.52 inch),Android 13,64 GB,...,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,Neutral,Neutral,POCO,Neutral,Neutral
4,4,"POCO C51 (Royal Blue, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0DXKIN,Royal Blue,16.56 cm (6.52 inch),Android 13,64 GB,...,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,Neutral,Neutral,POCO,Neutral,Neutral


In [42]:
df2.drop(columns='Unnamed: 0',index=1)

,Mobile Name,Images,Price,In The Box,Model Number,Color,Display Size,Operating System,Internal Storage,RAM,...,Star Wise Rating,Total_Ratings,Total_Reviews,Review Title,REVIEW BODY,DL Sentiment Pred,ML Sentiment Pred,Company Name,DL Sentiment Prediction,ML Sentiment Prediction
0,"POCO M6 Pro 5G (Power Black, 128 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹12,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPSIN,Power Black,17.25 cm (6.79 inch),Android 13,128 GB,6 GB,...,"['799', '272', '81', '29', '98']","1,279",100,"['Good quality product', 'Really Nice', 'Value...",best phone in this price range thank you flipk...,Positive,Positive,POCO,Neutral,Neutral
2,"POCO M6 Pro 5G (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹10,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPRIN,Power Black,17.25 cm (6.79 inch),Android 13,64 GB,4 GB,...,"['3,585', '896', '261', '112', '269']",NAN,NAN,"['Worth the money', 'Wonderful', 'Awesome', 'M...",very nice phone premium look display quality,Positive,Positive,POCO,Positive,Positive
3,"POCO C51 (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0E6DIN,Power Black,16.56 cm (6.52 inch),Android 13,64 GB,4 GB,...,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,Neutral,Neutral,POCO,Neutral,Neutral
4,"POCO C51 (Royal Blue, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0DXKIN,Royal Blue,16.56 cm (6.52 inch),Android 13,64 GB,4 GB,...,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,Neutral,Neutral,POCO,Neutral,Neutral
5,"REDMI 12 (Moonstone Silver, 128 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹11,499","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EQLIN,Moonstone Silver,17.25 cm (6.79 inch),Android 13,128 GB,6 GB,...,"['1,858', '621', '236', '105', '326']","3,146",275,"['Just wow!', 'Terrific purchase', 'Really Nic...",the phone is too good i prefer to buy this pho...,Neutral,Neutral,REDMI,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,"OPPO F21s Pro 5G (Dawnlight Gold, 128 GB)",https://rukminim2.flixcart.com/image/312/312/l...,"₹11,999","Handset, Simcard tool, Adaptor, USB cable, Qui...",CPH2455,Dawnlight Gold,16.33 cm (6.43 inch),Android Q Android 11,128 GB,8 GB,...,"['734', '216', '67', '30', '125']","1,172",76,"['Great product', 'Worth every penny', 'Value-...",super phone and am so impressed with this than...,Positive,Positive,OPPO,Neutral,Neutral
946,"LAVA Blaze Pro 6GB+128GB (Glass Green, 128 GB)",https://rukminim2.flixcart.com/image/312/312/k...,"₹12,347","Handset, Charger, USB Type C Cable, Back Cover...",LZX404,Glass Green,16.51 cm (6.5 inch),Android 12,128 GB,6 GB,...,"['26', '14', '5', '0', '5']",50,2,"['Nice', 'Classy product']",m very disappointed when i played apart from s...,Neutral,Neutral,LAVA,Neutral,Neutral
947,"REDMI Note 9 (Arctic White, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹16,499","Handset, Power Adapter, USB Type-C Cable, SIM ...",M2003J15SI / MZB9365IN / MZB082EIN,Arctic White,16.59 cm (6.53 inch),Android 10,64 GB,4 GB,...,"['75,903', '28,906', '9,835', '3,423', '6,759']","1,24,826","9,420","['Worth the money', 'Nice', 'Really Nice', 'Te...",good product good nice camera,Positive,Positive,REDMI,Neutral,Neutral
948,"SAMSUNG Galaxy A23 (Black, 128 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹19,999","Handset,User Manual,Sim Ejection Pin,Travel Ad...",SM-A235FZKHINS,Black,16.76 cm (6.6 inch),Android 12,128 GB,6 GB,...,"['642', '218', '104', '34', '93']","1,091",66,"['Simply awesome', 'Brilliant', 'Terrifi

In [43]:
ls=df2[df2['DL Sentiment Prediction']=='Neutral'].index

In [44]:
len(ls)

663

In [45]:
ls=df2[df2['DL Sentiment Prediction']=='Negative'].index

In [46]:
len(ls)

107

In [54]:
df2=df2.drop(columns=['DL Sentiment Pred','ML Sentiment Pred'],axis=1)

In [50]:
df2.to_csv('Final_Flipkart_Data.csv')

In [55]:
df2.head()

,Mobile Name,Images,Price,In The Box,Model Number,Color,Display Size,Operating System,Internal Storage,RAM,...,Battery Capacity,Rating,Star Wise Rating,Total_Ratings,Total_Reviews,Review Title,REVIEW BODY,Company Name,DL Sentiment Prediction,ML Sentiment Prediction
0,"POCO M6 Pro 5G (Power Black, 128 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹12,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPSIN,Power Black,17.25 cm (6.79 inch),Android 13,128 GB,6 GB,...,5000 mAh,4.3,"['799', '272', '81', '29', '98']","1,279",100,"['Good quality product', 'Really Nice', 'Value...",best phone in this price range thank you flipk...,POCO,Neutral,Neutral
2,"POCO M6 Pro 5G (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹10,999","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EPRIN,Power Black,17.25 cm (6.79 inch),Android 13,64 GB,4 GB,...,5000 mAh,4.4,"['3,585', '896', '261', '112', '269']",NAN,NAN,"['Worth the money', 'Wonderful', 'Awesome', 'M...",very nice phone premium look display quality,POCO,Positive,Positive
3,"POCO C51 (Power Black, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0E6DIN,Power Black,16.56 cm (6.52 inch),Android 13,64 GB,4 GB,...,5000 mAh,4.1,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,POCO,Neutral,Neutral
4,"POCO C51 (Royal Blue, 64 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹6,499","Handset, 10W Adapter, USB Cable, Sim Eject Too...",MZB0DXKIN,Royal Blue,16.56 cm (6.52 inch),Android 13,64 GB,4 GB,...,5000 mAh,4.1,"['33,513', '11,048', '5,212', '2,362', '6,415']","58,550","3,213","['Best in the market!', 'Simply awesome', 'Nic...",very good phone justify with the price good mo...,POCO,Neutral,Neutral
5,"REDMI 12 (Moonstone Silver, 128 GB)",https://rukminim2.flixcart.com/image/312/312/x...,"₹11,499","Handset, 22.5W Charger, USB Type-C Cable, Sim ...",MZB0EQLIN,Moonstone Silver,17.25 cm (6.79 inch),Android 13,128 GB,6 GB,...,5000 mAh,4.1,"['1,858', '621', '236', '105', '326']","3,146",275,"['Just wow!', 'Terrific purchase', 'Really Nic...",the phone is too good i prefer to buy this pho...,REDMI,Neutral,Neutral
